# Import libs

In [1]:
import re
import seaborn as sns
import os 
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
from tqdm.notebook import tqdm
import pickle
import plotly.express as px
import plotly.figure_factory as ff
from matplotlib import rcParams

tqdm.pandas()

sns.set_theme()
rcParams['figure.figsize'] = 15,8

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning:

The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version



In [2]:
!pip install vncorenlp

# Git clone

In [3]:
# Clone the entire repo.
!git clone https://github.com/Tuan-Lee-23/Vietnamese-Reactionary-Article-Classification-.git
%cd Vietnamese-Reactionary-Article-Classification-/
!ls

fatal: destination path 'Vietnamese-Reactionary-Article-Classification-' already exists and is not an empty directory.
/content/Vietnamese-Reactionary-Article-Classification-
analysis  data	demo  modeling	README.md


# Read pickles

In [4]:
%cd data/preprocessed/
! wget 'https://github.com/vncorenlp/VnCoreNLP/archive/v1.1.1.zip' -O ./models.$$ && unzip -o ./models.$$ && rm -r ./models.$$.

/content/Vietnamese-Reactionary-Article-Classification-/data/preprocessed
--2021-07-28 13:59:46--  https://github.com/vncorenlp/VnCoreNLP/archive/v1.1.1.zip
Resolving github.com (github.com)... 192.30.255.113
Connecting to github.com (github.com)|192.30.255.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/vncorenlp/VnCoreNLP/zip/v1.1.1 [following]
--2021-07-28 13:59:46--  https://codeload.github.com/vncorenlp/VnCoreNLP/zip/v1.1.1
Resolving codeload.github.com (codeload.github.com)... 140.82.114.9
Connecting to codeload.github.com (codeload.github.com)|140.82.114.9|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 143955573 (137M) [application/zip]
Saving to: ‘./models.201’

./models.201        100%[===================>] 137.29M  39.1MB/s    in 3.5s    

2021-07-28 13:59:50 (39.1 MB/s) - ‘./models.201’ saved [143955573/143955573]

Archive:  ./models.201
3b5c2aac53bb1fda0502d89ab8085b712e299fab
  inflat

## Read and write pickle function

In [5]:
def read_all_csv(dir):
    ls_df = []
    for data in os.listdir(dir):
        print(data)
        # try:
        df = pd.read_csv(dir + data)
        ls_df.append(df)
        # except:
            # pass

    final_df = pd.concat(ls_df)
    final_df.drop_duplicates('content', inplace = True)
    return final_df

df_1 = read_all_csv('./1/')
df_0 = read_all_csv('./0/')

df = pd.concat([df_1, df_0])

sbtn.csv
phamminhvu.csv
khoi8406.csv
thanhnienconggiao.csv
chantroimoi.csv
giaochucchuvanan.csv
rfa_fixed.csv
viettan_fb.csv
bpsos.csv
viettan.csv
vnexpress_1.csv
vnexpress_0.csv
vnexpress_2.csv
tinnoichinh.csv


In [6]:
df = df[df['avg_length'] < 20000]

# Remove abbreviation + teencode

## Read csv

In [7]:
abbreviation = pd.read_csv('./resources/abbreviation.csv')
abbreviation = abbreviation[['viet tat_x', 'mean']]
abbreviation['len'] = abbreviation['mean'].apply(lambda x: len(x.split()))
abbreviation = abbreviation[abbreviation['len'] > 2]

## Add them tu khoa

In [8]:
abbreviation = dict(zip(abbreviation['viet tat_x'], abbreviation['mean']))

abbreviation['csvn'] = 'Cộng sản Việt Nam'
abbreviation['đcsvn'] = 'Đảng Cộng sản Việt Nam'
abbreviation['xhcn'] = 'Xã hội Chủ nghĩa'
abbreviation['cntb'] = 'Chủ nghĩa Tư bản'
abbreviation['cs'] = 'Cộng sản'

## Replace key - value

In [9]:
def replace_non_case(df):
    count = 0
    for pat, repl in abbreviation.items():
        df['content'] = df['content'].str.replace(pat, repl, case = False, regex = False)
        
        # Progress printing
        if count % 10 == 0:
            print(f"count: {count}")
        count += 1
    

    return df

temp = df.copy()
temp = replace_non_case(temp)

count: 0
count: 10
count: 20
count: 30
count: 40
count: 50
count: 60
count: 70
count: 80
count: 90
count: 100
count: 110
count: 120
count: 130
count: 140
count: 150
count: 160
count: 170
count: 180
count: 190
count: 200
count: 210
count: 220
count: 230
count: 240
count: 250
count: 260
count: 270
count: 280
count: 290
count: 300
count: 310
count: 320
count: 330
count: 340
count: 350
count: 360
count: 370
count: 380
count: 390
count: 400
count: 410
count: 420
count: 430
count: 440
count: 450
count: 460
count: 470
count: 480
count: 490
count: 500
count: 510
count: 520
count: 530
count: 540
count: 550
count: 560
count: 570
count: 580
count: 590
count: 600
count: 610
count: 620
count: 630
count: 640
count: 650
count: 660
count: 670
count: 680
count: 690
count: 700
count: 710
count: 720
count: 730


## Số dòng được thay thế từ viết tắt

In [10]:
sum(temp['content'] != df['content'])

25145

## Save as csv

In [11]:
# df = temp
temp.to_csv('df_abv.csv')

In [12]:
df = temp

# Word segmentation

In [13]:
from vncorenlp import VnCoreNLP

VNCOR = './VnCoreNLP-1.1.1/VnCoreNLP-1.1.1.jar'
segmenter = VnCoreNLP(VNCOR, annotators= 'wseg')

In [14]:
# txt = 'học sinh học sinh học'
# print(segmenter.tokenize(txt))



def sen_word_seg(sample):
    """ Viblo - phoBERT """
    
    splits = sample.strip().split('\n')
    text = ' '.join(splits)

    # return
    text = segmenter.tokenize(text)
    text = ' '.join([' '.join(x) for x in text])
    
    return text


df['sen_word_seg'] = df['content'].progress_apply(sen_word_seg)

## Save to csv

In [15]:
df.to_csv('preprocessed.csv')